In [ ]:
from functions import *
from imports import *
from constants import *

In [ ]:
data_dict = {}

In [ ]:
driver  = create_driver()

In [ ]:
data_dict["urls"] ={}
data_dict["urls"]["Main Page"] = driver.current_url

In [ ]:
wait = WebDriverWait(driver=driver, timeout=3)
# Créer une instance d'ActionChains pour interagir avec l'élément
actions = ActionChains(driver)

# Connection

In [ ]:
button_mi_cuenta = driver.find_element(By.XPATH, 
                                       "//a[normalize-space()='Mi cuenta']")
if button_mi_cuenta.is_enabled():
    button_mi_cuenta.click()
    time_sleep()
    data_dict["urls"]["Connection Page"] = driver.current_url


email_zone = driver.find_element(By.XPATH, 
        "//form[@action='https://renzo.com.co/mi-cuenta/?action=login']//input[@id='username']")
email_zone.send_keys(my_email)
time_sleep()
password_zone = driver.find_element(By.XPATH, 
    "//form[@action='https://renzo.com.co/mi-cuenta/?action=login']//input[@id='password']")
password_zone.send_keys(my_password)
time_sleep()
recordame = driver.find_element(By.XPATH, 
        "//form[@action='https://renzo.com.co/mi-cuenta/?action=login']//input[@name='rememberme']")
if recordame.is_enabled() and not recordame.is_selected():
    recordame.click()
time_sleep()
ingresar = driver.find_element(By.XPATH,
    "//form[@action='https://renzo.com.co/mi-cuenta/?action=login']//button[@name='login'][normalize-space()='Ingresar']")
if ingresar.is_enabled() and not ingresar.is_selected():
    ingresar.click()
    WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.TAG_NAME, "body"))
    )
    time_sleep()    
    driver.get(data_dict["urls"]["Main Page"])


# Menu Nav-Bar

In [ ]:
main_menu = find_element(driver,"id", "menu-header-1")
choices = find_elements(main_menu, 'tag', 'a')
print(len(choices))

nav_menu = find_element(driver, 'xpath',"//div[contains(@class,'main-nav')]")
choice_hombre = WebDriverWait(nav_menu, 10).until(
    EC.visibility_of_element_located((By.XPATH, "//li[contains(@class,'menu-item menu-item-type-taxonomy menu-item-object-product_cat menu-item-has-children menu-item-8283 menu-item-design-default item-event-hover')]"))
)

# Effectuer l'action de survol
actions.move_to_element(choice_hombre).perform()
choice_ninos = find_element(nav_menu, 'xpath', 
"//li[contains(@class,'menu-item menu-item-type-post_type menu-item-object-page menu-item-has-children menu-item-8352 menu-item-design-default item-event-hover')]")

choice_promos = find_element(nav_menu, 'xpath', 
"//li[contains(@class,'color-sale-menu menu-item menu-item-type-taxonomy menu-item-object-product_cat menu-item-has-children menu-item-133578 menu-item-design-default item-event-hover')]")

choice_tiendas = find_element(nav_menu, 'xpath', 
"//li[contains(@class,'menu-item menu-item-type-post_type menu-item-object-page menu-item-8563 menu-item-design-default item-event-hover')]")
button_search = find_element(driver, "xpath", "//i[contains(@class,'fa-search')]")

button_wishlist = find_element(nav_menu,"xpath", 
                               "//a[contains(@href,'https://renzo.com.co/wishlist')]")
button_basket = find_element(nav_menu,"xpath","//a[contains(@href, 'https://renzo.com.co/carrito/')]")

# Query

In [ ]:
category_wanted = ["hombre", "promo"]
articles_wanted = ["pantalon"]
simple_search_query =[]

# Hombres

In [ ]:
list_hombres_articles = ['CAMISAS', 'LINOS', 'POLOS/T-SHIRT', 'PANTALONES', 'JEANS', 'BERMUDAS', 'PANTALONETAS',
                         'CORREAS', 'ZAPATOS']


time_sleep()
dict_hombres = {}

for item in list_hombres_articles:
    try:
        element = find_element(choice_hombre, 'xpath', f".//a[contains(text(), '{item}')]")
        dict_hombres[item]['element'] = element
        print(f"[OK] Élément trouvé pour {item}")
    except Exception as e:
        print(f"[!!] Élément manquant pour {item} : {e}")


# Ninos

In [ ]:
list_ninos_articles = ['CAMISAS', 'LINOS', 'POLOS/T-SHIRT', 'PANTALONES', 'JEANS', 'BERMUDAS',
                       'PANTALONETAS', 'CORREAS', 'ZAPATOS', 'SET', 'ACCESORIOS', 'Outfit']

dict_ninos = {}

for item in list_ninos_articles:
    try:
        element = find_element(choice_ninos, 'xpath', f".//a[contains(text(), '{item}')]")
        dict_ninos[item] = element
        print(f"[OK] Élément trouvé pour {item}")
    except Exception as e:
        print(f"[!!] Élément manquant pour {item} : {e}")


# Promos

In [ ]:
promos_page_links = [
    'https://renzo.com.co/categoria-producto/ofertas-ropa-hombre-ninos/sale-renzo/',
    'https://renzo.com.co/categoria-producto/ofertas-ropa-hombre-ninos/sale-bambino/'
]

for item in promos_page_links:
    try:
        promos_hombres = find_element(choice_promos, 'xpath', 
            f".//a[contains(@href, '{item}')]")
        print("[OK] Élément trouvé pour promos_hombres")
    except Exception as e:
        print(f"[!!] Élément manquant pour promos_hombres : {e}")

    try:
        promos_ninos = find_element(choice_promos, 'xpath', 
            f".//a[contains(@href, '{item}')]")
        print("[OK] Élément trouvé pour promos_ninos")
    except Exception as e:
        print(f"[!!] Élément manquant pour promos_ninos : {e}")


In [ ]:
# Passe la souris sur HOMBRE pour voir les sous-menus
actions.move_to_element(choice_hombre).perform()
time.sleep(1)  # Temps pour que le sous-menu apparaisse

# Search Mechanics